# Time series forecasting

In this notebook you will learn how to perform time series forecasting. In particular, we will develop regression-based forecasting. The content of this notebook is mostly based on the examples of the text book.

> (c) 2019 Galit Shmueli, Peter C. Bruce, Peter Gedeck 
>
> Code included in
>
> _Data Mining for Business Analytics: Concepts, Techniques, and Applications in Python_ (First Edition) 
> Galit Shmueli, Peter C. Bruce, Peter Gedeck, and Nitin R. Patel. 2019.

As always, let's get started by loading all the required libraries:

In [ ]:
import dmba
import warnings

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import statsmodels.formula.api as sm

from statsmodels.tsa import tsatools
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics import tsaplots
from plot_utils import graph_layout
from plot_utils import single_graph_layout

%matplotlib inline
warnings.filterwarnings('ignore')

## Dataset

The very first step is to load and prepare the data for further analysis. We will use ridership data of the Amtrak railway company.

In [ ]:
df_ridership = dmba.load_data('Amtrak.csv')
df_ridership.head()

In [ ]:
# Checking column types
df_ridership.info()

We will create a new column called `Date`, which is basically the Month column transformed to datetime type:

In [ ]:
df_ridership['Date'] = pd.to_datetime(df_ridership.Month, format='%d/%m/%Y')
df_ridership.head()

Let's now create a Pandas series that contains our ridership time series with monthly frequency data:

In [ ]:
df_ts = pd.Series(df_ridership.Ridership.values, index=df_ridership.Date, name='Ridership')
df_ts.index = pd.DatetimeIndex(df_ts.index, freq=df_ts.index.inferred_freq)
df_ts.head()

We can finally have a look at our time series using a line plot:

In [ ]:
ax = df_ts.plot(figsize=(10,5))
ax.set_xlabel('Time')
ax.set_ylabel('Ridership (in 000s)')
ax.set_ylim(1300, 2300)
plt.show()

The very last preparation step is to add a trend component to the time series for further modelling:

In [ ]:
df = tsatools.add_trend(df_ts, trend='ct')
df.head()

As you can see, we have a constant component, which will help us to fit the intercept, and a trend component, which will be used for the trend. The latter is basically equivalent to each timestep on the series. When we perform a linear fit, we cannot give time directly as a predictor, therefore we create a variable with consecutive numbers, which represents each timestep.


## Dataset partition

The following step is to divide the data into train and test sets. Remember that for time series we cannot use random sets, as this would create two time series with holes. Therefore we use the earlier periods for training and the later periods for validation.

In [ ]:
m_test = 36
m_train = len(df) - m_test

df_train = df[:m_train]
df_train.tail()

In [ ]:
# TODO: create a dataframe (df_test) with the test data


## Modelling trend

As we saw in the theoretical lecture, this time series has a U-shape trend, which indicates that a quadratic function is a good representation for the trend. We will use the `statsmodels` library to fit the trend, the syntax is quite similar to R:

In [ ]:
ridership_trend = sm.ols(formula='Ridership ~ trend + np.square(trend)', data=df_train).fit()

We used two predictors (t, t^2). Let's now calculate the residuals, which is the difference between the fit and the actual values, and visualize the results, which also include forecasts for the test set:

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9, 7.5))

ridership_trend.predict(df_train).plot(ax=axes[0], color='C1')
ridership_trend.predict(df_test).plot(ax=axes[0], color='C1', linestyle='dashed')
    
residual = df_train.Ridership - ridership_trend.predict(df_train)
residual.plot(ax=axes[1], color='C1')
residual = df_test.Ridership - ridership_trend.predict(df_test)
residual.plot(ax=axes[1], color='C1', linestyle='dashed')

graph_layout(axes, df_train, df_test)

plt.show()

Is this a good fit? What do you see in the residuals?

-> We can see in the residuals that the trend was well modelled (U-shape removed), but we still see a seasonality pattern left.

Let's also have a look at the regression results:

In [ ]:
print(ridership_trend.summary())

## Modelling seasonality

As we discussed in the theoretical lesson, this time series has a monthly seasonal pattern. Let's zoom in to have a better look at it:

In [ ]:
ax = df_ts['1997':'1999'].plot(figsize=(10,5))
ax.set_xlabel('Time')
ax.set_ylabel('Ridership (in 000s)')
ax.set_ylim(1300, 2310)
plt.show()

Do you see that we can easily slice the time series using years, as the dates are used as index?
But coming back to the topic of seaslonality, we can model it by creating a categorical variable for the season. So let's see how it works:

In [ ]:
df = tsatools.add_trend(df_ts, trend='c')
df['Month'] = df.index.month
df.head()

Re-partitioning the data:

In [ ]:
df_train = df[:m_train]
df_test = df[m_train:]

Let's now fit the seasonality:

In [ ]:
ridership_season = sm.ols(formula='Ridership ~ C(Month)', data=df_train).fit()

Now we can calculate the residuals and visualize the results:

In [ ]:
# TODO: calculate residuals and visualize results


And now? What do you think of this fit?

-> The residuals now displays the U-shaped trend.

It is also useful to check the regression results:

In [ ]:
# TODO: print the regression results


## Modelling trend and seasonality

We can see that we need to add both the trend and seasonality to get a proper model. So let's do it, but before we will do some quick data preparation:

In [ ]:
# Preparing data
df = tsatools.add_trend(df_ts, trend='ct')
df['Month'] = df.index.month

# Partition the data
df_train = df[:m_train]
df_test = df[m_train:]

df.head()

Let's now train a model with both trend and seasonality:

In [ ]:
# TODO: create and fit a model with both trend and seasonality


Calculate the residuals and plot the results:

In [ ]:
# TODO: calculate the residuals and plot the results


Finally, you can print the regression results:

In [ ]:
# TODO: check model summary


What do you think of this model? Can it still be improved?

-> The current model is much better, but it can still be improved!


## Autocorrelation

We saw in the theoretical lecture that autocorrelation is basically a measure of the correlation of a series and itself, more precisely its lagged versions. Let's have a look at the autocorrelation plot for this time series:

In [ ]:
tsaplots.plot_acf(df_train['1991-01-01':'1993-01-01'].Ridership)
plt.show()

We can see a strong correlation at lag-6, which is statistically significant as it falls outside of the blue region. The high correlation at lag-6 is basically due to the high summers and low winters pattern. Let's now check the autocorrelation of the residuals:

In [ ]:
# TODO: assign your previous model (with trend and seasonality) to the ridership_trend_season variable


In [ ]:
residual = df_train.Ridership - ridership_trend_season.predict(df_train)
tsaplots.plot_acf(residual)
plt.xlim(-1, 13)
plt.show()

We can see positive correlation for all the lags, with lags 1 to 3 outside the 95% confidence interval, which indicates that the model can still be improved. That was also visible in the residuals. We can improve this model using a second-level forecast. A way of doing that is to use our original model and add a correction term. This correction term can be estimated by running an autoregressive model on the residuals. So let's do that:

In [ ]:
train_res_arima = ARIMA(ridership_trend_season.resid, order=(1, 0, 0), freq='MS').fit()

We trained an autoregressive model of order 1 on the residuals of the original model. Let's have a look at the coefficients:

In [ ]:
print(pd.DataFrame({'coef': train_res_arima.params, 'std err': train_res_arima.bse}))

Let's visualize as a line plot:

In [ ]:
ax = ridership_trend_season.resid.plot(figsize=(9,4))
train_res_arima.fittedvalues.plot(ax=ax)
single_graph_layout(ax, [-250, 250], df_train, df_test)
plt.show()

The forecast for the error on April 2001 can be obtained by:

In [ ]:
train_res_arima.forecast(1)

And the forecast for ridership on April 2001 can be obtained by:

In [ ]:
ridership_trend_season.predict(df_test).head(1)

So our second-level forecast for April 2001 would be:

In [ ]:
ridership_trend_season.predict(df_test).values[0] + train_res_arima.forecast(1)[0]

And the observed ridership is:

In [ ]:
df_ts['2001-04-01']

As you can see, the second-level forecast provides a better estimation. Let's have a look at the autocorrelation plot for the residuals-of-residuals:

In [ ]:
tsaplots.plot_acf(train_res_arima.resid)
plt.xlim(-1, 13)
plt.show()

We can see that autocorrelation was removed, which indicates a good model. Now you know how to build a regression-based forecast and how to use autoregressive models to improve forecasts. Notice, however, that using autoregressive models on the residuals can provide a good estimation for the error only for short-term forecasts. The problem here is that we need to know the error of past values to forecast the error of future values. If we look too far into the future, we would need to use forecasts for these errors, which would add too much uncertainty.